In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
#load Amfar opioid and HIV data, add county code
opiod_df = pd.read_table('data/amfAR/countydata.tsv',header=0)
opiod_df['county_code'] = opiod_df.STATEFP*1000 + opiod_df.COUNTYFP # build a county code column
opiod_df['county_code'] = opiod_df.county_code.astype(int)

In [3]:
opiod_df.head()

,STATEFP,COUNTYFP,YEAR,INDICATOR,VALUE,COUNTY,STATE,STATEABBREVIATION,county_code
0,1,1,2017,bup_phys,2.0,Autauga County,Alabama,AL,1001
1,1,3,2017,bup_phys,13.0,Baldwin County,Alabama,AL,1003
2,1,5,2017,bup_phys,0.0,Barbour County,Alabama,AL,1005
3,1,7,2017,bup_phys,0.0,Bibb County,Alabama,AL,1007
4,1,9,2017,bup_phys,1.0,Blount County,Alabama,AL,1009


In [4]:
#make changes to the amfar dataframe
#convert from long to wide format
index_lst = ['county_code', 'COUNTY', 'STATEABBREVIATION', 'YEAR']
col_lst = ['INDICATOR']
opiod_df_wide = opiod_df.pivot_table(values='VALUE', index=index_lst, columns=col_lst).reset_index()

In [5]:
opiod_df_wide.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,CDC_consult,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,...,drugdeathrate_est,drugdeaths,drugdep,mme_percap,nonmedpain,num_SSPs,partD30dayrxrate,pctunins,pctunmetneed,vulnerable_rank
0,1001,Autauga County,AL,1999,NaN,NaN,NaN,NaN,NaN,NaN,...,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,Autauga County,AL,2000,NaN,NaN,NaN,NaN,NaN,NaN,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,Autauga County,AL,2001,NaN,NaN,NaN,NaN,NaN,NaN,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001,Autauga County,AL,2002,NaN,NaN,NaN,NaN,NaN,NaN,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001,Autauga County,AL,2003,NaN,NaN,NaN,NaN,NaN,NaN,...,4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Focus on just the HIV related data, from 2008 onward
opiod_df_wide = opiod_df_wide[opiod_df_wide['YEAR'] >= 2008] # subset for years that have hiv data
cols_to_drop = ['CDC_consult', 'vulnerable_rank']
#, 'num_SSPs', 'bup_phys', 'drugdep', 'pctunmetneed', 'nonmedpain']
opiod_df_wide.drop(cols_to_drop, axis=1, inplace=True) # drop unnecessary columns

In [7]:
opiod_df_wide.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,Med_AMAT_fac,...,drugdeathrate,drugdeathrate_est,drugdeaths,drugdep,mme_percap,nonmedpain,num_SSPs,partD30dayrxrate,pctunins,pctunmetneed
9,1001,Autauga County,AL,2008,0.0,8.0,19.8,NaN,NaN,0.0,...,NaN,6.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1001,Autauga County,AL,2009,0.0,7.0,17.1,NaN,NaN,0.0,...,NaN,6.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1001,Autauga County,AL,2010,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,8.1,NaN,NaN,1050.8,NaN,NaN,NaN,NaN,NaN
12,1001,Autauga County,AL,2011,0.0,7.0,15.5,NaN,NaN,0.0,...,NaN,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1001,Autauga County,AL,2012,0.0,7.0,15.5,NaN,NaN,0.0,...,NaN,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
fill_nan_cols = ['HIVdiagnoses', 'HIVincidence', 'HIVprevalence',
                 'PLHIV', 'drugdeathrate', 'drugdeaths']
opiod_df_wide[fill_nan_cols] = opiod_df_wide[fill_nan_cols].fillna(0) #fill NaNs for suppressed data with zeroes

In [9]:
opiod_df_wide.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,Med_AMAT_fac,...,drugdeathrate,drugdeathrate_est,drugdeaths,drugdep,mme_percap,nonmedpain,num_SSPs,partD30dayrxrate,pctunins,pctunmetneed
9,1001,Autauga County,AL,2008,0.0,8.0,19.8,0.0,NaN,0.0,...,0.0,6.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1001,Autauga County,AL,2009,0.0,7.0,17.1,0.0,NaN,0.0,...,0.0,6.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1001,Autauga County,AL,2010,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,8.1,0.0,NaN,1050.8,NaN,NaN,NaN,NaN,NaN
12,1001,Autauga County,AL,2011,0.0,7.0,15.5,0.0,NaN,0.0,...,0.0,8.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1001,Autauga County,AL,2012,0.0,7.0,15.5,0.0,NaN,0.0,...,0.0,8.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Subset data to 2015
opiod_df_15 = opiod_df_wide[opiod_df_wide['YEAR'] == 2015]

In [11]:
opiod_df_15.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,Med_AMAT_fac,...,drugdeathrate,drugdeathrate_est,drugdeaths,drugdep,mme_percap,nonmedpain,num_SSPs,partD30dayrxrate,pctunins,pctunmetneed
16,1001,Autauga County,AL,2015,0.0,5.0,10.9,225.5,1.0,0.0,...,0.0,10.1,0.0,NaN,1084.0,NaN,NaN,2.34,10.1,NaN
35,1003,Baldwin County,AL,2015,0.0,15.0,8.7,163.9,4.0,0.0,...,21.1,20.1,43.0,NaN,1148.5,NaN,NaN,2.56,13.0,NaN
54,1005,Barbour County,AL,2015,0.0,0.0,0.0,436.0,1.0,0.0,...,0.0,6.1,0.0,NaN,577.6,NaN,NaN,1.46,15.5,NaN
73,1007,Bibb County,AL,2015,0.0,0.0,0.0,191.9,0.0,0.0,...,0.0,14.1,0.0,NaN,563.0,NaN,NaN,0.39,9.7,NaN
92,1009,Blount County,AL,2015,0.0,5.0,10.4,95.4,1.0,0.0,...,0.0,16.1,11.0,NaN,459.0,NaN,NaN,1.22,11.6,NaN


In [12]:
pd.options.mode.chained_assignment = None  # default='warn', get rid of annoying warning associated with dropping below
# drop columns having no 2015 data, will be extraploated from following years later
opiod_df_15.drop(['num_SSPs', 'bup_phys', 'drugdep', 'pctunmetneed', 'nonmedpain'], axis=1, inplace=True)

In [13]:
opiod_df_15.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,Med_AMAT_fac,...,Population,SA_fac,SMAT_fac,TMAT_fac,drugdeathrate,drugdeathrate_est,drugdeaths,mme_percap,partD30dayrxrate,pctunins
16,1001,Autauga County,AL,2015,0.0,5.0,10.9,225.5,1.0,0.0,...,55035.0,2.0,0.0,0.0,0.0,10.1,0.0,1084.0,2.34,10.1
35,1003,Baldwin County,AL,2015,0.0,15.0,8.7,163.9,4.0,0.0,...,203690.0,2.0,0.0,0.0,21.1,20.1,43.0,1148.5,2.56,13.0
54,1005,Barbour County,AL,2015,0.0,0.0,0.0,436.0,1.0,0.0,...,26270.0,1.0,0.0,0.0,0.0,6.1,0.0,577.6,1.46,15.5
73,1007,Bibb County,AL,2015,0.0,0.0,0.0,191.9,0.0,0.0,...,22561.0,1.0,1.0,0.0,0.0,14.1,0.0,563.0,0.39,9.7
92,1009,Blount County,AL,2015,0.0,5.0,10.4,95.4,1.0,0.0,...,57676.0,1.0,0.0,0.0,0.0,16.1,11.0,459.0,1.22,11.6


In [14]:
# get esimates for num_SSPs, bug_phys, drug_dep, pctunmetneed, and nonmedpain from following years

#subset opioid related data from one year only
#number of needle exchange programs (num_SSPs)
opiod_df_wide_17 = opiod_df_wide[opiod_df_wide['YEAR'] == 2017]
df_num_SSP = opiod_df_wide_17[['num_SSPs', 'county_code']]

#number of doctors licensed to rx Buprenorphine (bup_phys)
df_bup_phys = opiod_df_wide_17[['bup_phys', 'county_code']]

#percent with drug dependency (drug_dep)
opiod_df_wide_16 = opiod_df_wide[opiod_df_wide['YEAR'] == 2016]
df_drugdep = opiod_df_wide_16[['drugdep', 'county_code']]

#percent unmet drug treatment need (pctunmetneed)
df_pctunmetneed = opiod_df_wide_16[['pctunmetneed', 'county_code']]

#percent taken pain meds for nonmedical use (nonmedpain)
df_nonmedpain = opiod_df_wide_16[['nonmedpain', 'county_code']]

In [15]:
# merge these values back into 2015 dataframe
#merge opioid related data back to the 2015 dataframe
opiod_df_15 = opiod_df_15.merge(df_num_SSP, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_bup_phys, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_drugdep, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_pctunmetneed, on='county_code')
opiod_df_15 = opiod_df_15.merge(df_nonmedpain, on='county_code')

In [16]:
opiod_df_15.head()

INDICATOR,county_code,COUNTY,STATEABBREVIATION,YEAR,AMAT_fac,HIVdiagnoses,HIVincidence,HIVprevalence,MH_fac,Med_AMAT_fac,...,drugdeathrate_est,drugdeaths,mme_percap,partD30dayrxrate,pctunins,num_SSPs,bup_phys,drugdep,pctunmetneed,nonmedpain
0,1001,Autauga County,AL,2015,0.0,5.0,10.9,225.5,1.0,0.0,...,10.1,0.0,1084.0,2.34,10.1,0.0,2.0,2.56,95.70,5.12
1,1003,Baldwin County,AL,2015,0.0,15.0,8.7,163.9,4.0,0.0,...,20.1,43.0,1148.5,2.56,13.0,0.0,13.0,2.54,91.34,5.27
2,1005,Barbour County,AL,2015,0.0,0.0,0.0,436.0,1.0,0.0,...,6.1,0.0,577.6,1.46,15.5,0.0,0.0,2.54,91.34,5.27
3,1007,Bibb County,AL,2015,0.0,0.0,0.0,191.9,0.0,0.0,...,14.1,0.0,563.0,0.39,9.7,0.0,0.0,2.58,91.86,5.62
4,1009,Blount County,AL,2015,0.0,5.0,10.4,95.4,1.0,0.0,...,16.1,11.0,459.0,1.22,11.6,0.0,1.0,2.58,91.86,5.62


In [17]:
#load Men who have sex with men (MSM) estimate data
msm_df = pd.read_csv("data/CAMP/US_MSM_Estimates_Data_2013.csv")    #load the data
msm_df['county_code'] = msm_df.STATEFP*1000 + msm_df.COUNTYFP  # build a county code column
msm_df['county_code'] = msm_df.county_code.astype(int)
msm_df['%msm12month'] = 100 * (msm_df.MSM12MTH / msm_df.ADULTMEN) # build a %MSM within last 12 months column
msm_df['%msm5yr'] = 100 * (msm_df.MSM5YEAR / msm_df.ADULTMEN)     # build a %MSM within last 5years column

In [18]:
msm_df.head()

,REGCODE,DIVCODE,STATEFP,COUNTYFP,CSACODE,CBSACODE,METDCODE,METMICSA,CENTOUTL,ADULTMEN,MSM12MTH,MSM5YEAR,county_code,%msm12month,%msm5yr
0,3,6,1,1,NaN,33860.0,NaN,1.0,NaN,19410,333,514,1001,1.715611,2.648120
1,3,6,1,3,380.0,19300.0,NaN,1.0,1.0,69724,925,1429,1003,1.326659,2.049509
2,3,6,1,5,NaN,NaN,NaN,NaN,NaN,11567,82,127,1005,0.708913,1.097951
3,3,6,1,7,142.0,13820.0,NaN,1.0,2.0,9508,119,184,1007,1.251578,1.935212
4,3,6,1,9,142.0,13820.0,NaN,1.0,2.0,21368,601,928,1009,2.812617,4.342943


In [19]:
cols_to_drop = ['REGCODE', 'DIVCODE', 'STATEFP', 'COUNTYFP', 'CSACODE', 
                'CBSACODE','METDCODE', 'METMICSA', 'CENTOUTL']
msm_df.drop(cols_to_drop, axis=1, inplace=True) #drop all unneeded columns
msm_df.head()

,ADULTMEN,MSM12MTH,MSM5YEAR,county_code,%msm12month,%msm5yr
0,19410,333,514,1001,1.715611,2.648120
1,69724,925,1429,1003,1.326659,2.049509
2,11567,82,127,1005,0.708913,1.097951
3,9508,119,184,1007,1.251578,1.935212
4,21368,601,928,1009,2.812617,4.342943


In [20]:
#unemplyment data
df_employment = pd.read_csv("data/ACS_14_5YR_employment/ACS_14_5YR_S2301_with_ann.csv", 
                            encoding = "ISO-8859-1", skiprows=1)
df_employment = df_employment[['Id2', 'Unemployment rate; Estimate; Population 16 years and over']]
df_employment.columns = ['county_code', 'unemployment_rate']
df_employment.head()

,county_code,unemployment_rate
0,1001,8.5
1,1003,8.6
2,1005,14.2
3,1007,10.9
4,1009,9.3


In [21]:
#poverty data
df_poverty = pd.read_csv("data/ACS_14_5YR_poverty/ACS_14_5YR_S1701_with_ann.csv", 
                         encoding = "ISO-8859-1", skiprows=1)
df_poverty = df_poverty[['Id2', 'Percent below poverty level; Estimate; Population for whom poverty status is determined']]
df_poverty.columns = ['county_code', 'poverty_rate']
df_poverty.head()

,county_code,poverty_rate
0,1001,12.8
1,1003,13.8
2,1005,24.1
3,1007,17.0
4,1009,17.3


In [22]:
#income data
df_income = pd.read_csv("data/ACS_14_5YR_income/ACS_14_5YR_S1901_with_ann.csv", 
                        encoding = "ISO-8859-1", skiprows=1)
df_income = df_income[['Id2', 'Households; Estimate; Total']]
df_income.columns = ['county_code', 'household_income']
df_income.head()

,county_code,household_income
0,1001,20304
1,1003,73058
2,1005,9145
3,1007,7078
4,1009,20934


In [62]:
#merge asfAR hiv/opioid data with CAMP MSM data
df_main = opiod_df_15.merge(msm_df, on='county_code')

#merge in ACS data
df_main = df_main.merge(df_employment, on='county_code')
df_main = df_main.merge(df_poverty, on='county_code')
df_main = df_main.merge(df_income, on='county_code')

In [63]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3140 entries, 0 to 3139
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county_code        3140 non-null   int64  
 1   COUNTY             3140 non-null   object 
 2   STATEABBREVIATION  3140 non-null   object 
 3   YEAR               3140 non-null   int64  
 4   AMAT_fac           3140 non-null   float64
 5   HIVdiagnoses       3140 non-null   float64
 6   HIVincidence       3140 non-null   float64
 7   HIVprevalence      3140 non-null   float64
 8   MH_fac             3140 non-null   float64
 9   Med_AMAT_fac       3140 non-null   float64
 10  Med_MH_fac         3140 non-null   float64
 11  Med_SA_fac         3140 non-null   float64
 12  Med_SMAT_fac       3140 non-null   float64
 13  Med_TMAT_fac       3140 non-null   float64
 14  PLHIV              3140 non-null   float64
 15  Population         3140 non-null   float64
 16  SA_fac             3140 

In [64]:
df_main.shape

(3140, 38)

In [92]:
df2 = df_main.dropna(axis=0, how='any')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2937 entries, 0 to 3139
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county_code        2937 non-null   int64  
 1   COUNTY             2937 non-null   object 
 2   STATEABBREVIATION  2937 non-null   object 
 3   YEAR               2937 non-null   int64  
 4   AMAT_fac           2937 non-null   float64
 5   HIVdiagnoses       2937 non-null   float64
 6   HIVincidence       2937 non-null   float64
 7   HIVprevalence      2937 non-null   float64
 8   MH_fac             2937 non-null   float64
 9   Med_AMAT_fac       2937 non-null   float64
 10  Med_MH_fac         2937 non-null   float64
 11  Med_SA_fac         2937 non-null   float64
 12  Med_SMAT_fac       2937 non-null   float64
 13  Med_TMAT_fac       2937 non-null   float64
 14  PLHIV              2937 non-null   float64
 15  Population         2937 non-null   float64
 16  SA_fac             2937 

In [93]:

county_index = df2[['HIVincidence', 'COUNTY','STATEABBREVIATION', 'YEAR']]
y = df2['HIVincidence']
X = df2.drop(['HIVincidence', 'COUNTY','STATEABBREVIATION', 'YEAR', 'mme_percap', 'partD30dayrxrate','drugdep', 'pctunmetneed', 'nonmedpain'], axis=1)

In [94]:
X = sm.add_constant(X)

In [71]:
y

0       10.9
1        8.7
2        0.0
3        0.0
4       10.4
        ... 
3135     0.0
3136     0.0
3137     0.0
3138     0.0
3139     0.0
Name: HIVincidence, Length: 3140, dtype: float64

In [96]:
reg  = sm.OLS(y, X)
reg = reg.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           HIVincidence   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.269
Method:                 Least Squares   F-statistic:                     38.25
Date:                Fri, 31 Jul 2020   Prob (F-statistic):          6.31e-180
Time:                        13:16:25   Log-Likelihood:                -12031.
No. Observations:                2937   AIC:                         2.412e+04
Df Residuals:                    2907   BIC:                         2.430e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.9202      1.316     -1.460      0.144      -4.500       0.659
county_code         7.48e-07   1.86e-05      0.040      0.968   -3.56e-05    3.71e-05
AMAT_fac             -4.0652      1.828     -2.224      0.026      -7.649      -0.481
HIVdiagnoses          0.3416      0.019     17.564      0.000       0.303       0.380
HIVprevalence         0.0369      0.002     19.073      0.000       0.033       0.041
MH_fac                1.1760      0.370      3.174      0.002       0.450       1.902
Med_AMAT_fac          4.4758      2.067      2.165      0.030       0.422       8.529
Med_MH_fac           -1.0637      0.403     -2.641      0.008      -1.853      -0.274
Med_SA_fac           -0.3160      0.231     -1.365      0.172      -0.770       0.138
Med_SMAT_fac          1.6133      0.711      2.270      0.023       0.220       3.007
Med_TMAT_fac         -1.1374      1.016     -1.119      0.263      -3.130       0.855
PLHIV                -0.0112      0.001    -15.009      0.000      -0.013      -0.010
Population         1.484e-05   2.44e-05      0.609      0.542   -3.29e-05    6.26e-05
SA_fac                0.0965      0.132      0.731      0.465      -0.162       0.355
SMAT_fac             -0.6908      0.588     -1.174      0.240      -1.844       0.463
TMAT_fac              0.9385      0.803      1.169      0.242      -0.636       2.513
drugdeathrate         0.0265      0.035      0.758      0.448      -0.042       0.095
drugdeathrate_est     0.0590      0.047      1.264      0.206      -0.033       0.151
drugdeaths           -0.0580      0.017     -3.320      0.001      -0.092      -0.024
pctunins             -0.0213      0.066     -0.321      0.748      -0.152       0.109
num_SSPs              3.5498      0.822      4.320      0.000       1.939       5.161
bup_phys              0.0806      0.033      2.479      0.013       0.017       0.144
ADULTMEN             -0.0001   7.24e-05     -1.692      0.091      -0.000    1.95e-05
MSM12MTH              1.3507      0.978      1.381      0.167      -0.567       3.268
MSM5YEAR             -0.8744      0.633     -1.382      0.167      -2.115       0.366
%msm12month          -1.6060     38.180     -0.042      0.966     -76.468      73.256
%msm5yr               1.8516     24.704      0.075      0.940     -46.588      50.291
unemployment_rate     0.0569      0.106      0.536      0.592      -0.151       0.265
poverty_rate         -0.1244      0.064     -1.946      0.052      -0.250       0.001
household_income   5.275e-05   4.21e-05      1.253      0.210   -2.98e-05       0.000
==============================================================================
Omnibus:                     8268.951   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        350536795.939
Skew:   

In [102]:
feature_lst = [x for x in X.columns]

In [107]:
df2 = sm.add_constant(X)
feat_drop1 = ['const',
 'AMAT_fac',
 'HIVdiagnoses',
 'HIVprevalence',
 'MH_fac',
 'Med_AMAT_fac',
 'Med_MH_fac',
 'Med_SA_fac',
 'Med_SMAT_fac',
 'Med_TMAT_fac',
 'PLHIV',
 'SA_fac',
 'SMAT_fac',
 'TMAT_fac',
 'drugdeathrate',
 'drugdeathrate_est',
 'drugdeaths',
 'num_SSPs',
 'bup_phys',
 'ADULTMEN',
 'MSM12MTH',
 'MSM5YEAR',
 'unemployment_rate',
 'poverty_rate',
 'household_income']

In [108]:
X2 = df2[feat_drop1]

In [110]:
reg  = sm.OLS(y, X2)
reg = reg.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           HIVincidence   R-squared:                       0.273
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     45.51
Date:                Fri, 31 Jul 2020   Prob (F-statistic):          3.93e-181
Time:                        13:24:01   Log-Likelihood:                -12038.
No. Observations:                2937   AIC:                         2.413e+04
Df Residuals:                    2912   BIC:                         2.428e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.1324      0.964     -0.137      0.891      -2.024       1.759
AMAT_fac             -4.3802      1.819     -2.408      0.016      -7.946      -0.814
HIVdiagnoses          0.3395      0.019     17.444      0.000       0.301       0.378
HIVprevalence         0.0386      0.002     20.623      0.000       0.035       0.042
MH_fac                1.0644      0.367      2.904      0.004       0.346       1.783
Med_AMAT_fac          4.8063      2.060      2.333      0.020       0.767       8.846
Med_MH_fac           -0.9716      0.401     -2.421      0.016      -1.758      -0.185
Med_SA_fac           -0.2480      0.227     -1.093      0.274      -0.693       0.197
Med_SMAT_fac          1.4107      0.699      2.019      0.044       0.041       2.781
Med_TMAT_fac         -1.0220      1.014     -1.008      0.314      -3.010       0.966
PLHIV                -0.0112      0.001    -15.172      0.000      -0.013      -0.010
SA_fac                0.0449      0.124      0.361      0.718      -0.199       0.289
SMAT_fac             -0.5434      0.580     -0.937      0.349      -1.680       0.593
TMAT_fac              0.8926      0.803      1.112      0.266      -0.681       2.467
drugdeathrate         0.0384      0.035      1.110      0.267      -0.029       0.106
drugdeathrate_est     0.0570      0.046      1.232      0.218      -0.034       0.148
drugdeaths           -0.0545      0.017     -3.125      0.002      -0.089      -0.020
num_SSPs              3.6778      0.817      4.501      0.000       2.076       5.280
bup_phys              0.0743      0.031      2.378      0.017       0.013       0.136
ADULTMEN             -0.0001   3.63e-05     -2.816      0.005      -0.000    -3.1e-05
MSM12MTH              1.2382      0.777      1.593      0.111      -0.286       2.763
MSM5YEAR             -0.8014      0.503     -1.594      0.111      -1.788       0.185
unemployment_rate     0.0686      0.105      0.656      0.512      -0.136       0.273
poverty_rate         -0.1748      0.057     -3.067      0.002      -0.287      -0.063
household_income   7.346e-05   4.03e-05      1.824      0.068   -5.53e-06       0.000
==============================================================================
Omnibus:                     8276.181   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        352793955.545
Skew:                          35.983   Prob(JB):                         0.00
Kurtosis:                    1699.382   Cond. No.                     1.75e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.75e+06. This might indicate that there are
strong multicollin

In [138]:
'''
'MH_fac',
'Med_AMAT_fac',
'Med_MH_fac',
'Med_SA_fac',
'Med_SMAT_fac',
'Med_TMAT_fac',
'SMAT_fac',
'TMAT_fac',
'AMAT_fac',
 'bup_phys',
 'ADULTMEN',
 'MSM12MTH',
 'MSM5YEAR',
'''


feat_drop2 = ['const',
 'Population',
 'HIVdiagnoses',
 'HIVprevalence',
 'bup_phys',
 'PLHIV',
 'num_SSPs',
 'poverty_rate']
X3 = df2[feat_drop2]

In [139]:
reg  = sm.OLS(y, X3)
reg = reg.fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           HIVincidence   R-squared:                       0.260
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     146.8
Date:                Fri, 31 Jul 2020   Prob (F-statistic):          4.93e-186
Time:                        14:42:21   Log-Likelihood:                -12064.
No. Observations:                2937   AIC:                         2.414e+04
Df Residuals:                    2929   BIC:                         2.419e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             1.0979      0.804      1.366      0.172      -0.478       2.674
Population    -1.708e-05   2.43e-06     -7.040      0.000   -2.18e-05   -1.23e-05
HIVdiagnoses      0.2834      0.017     16.588      0.000       0.250       0.317
HIVprevalence     0.0389      0.002     22.677      0.000       0.036       0.042
bup_phys          0.0947      0.023      4.115      0.000       0.050       0.140
PLHIV            -0.0098      0.001    -15.797      0.000      -0.011      -0.009
num_SSPs          3.7913      0.791      4.796      0.000       2.241       5.341
poverty_rate     -0.1459      0.046     -3.187      0.001      -0.236      -0.056
==============================================================================
Omnibus:                     8319.343   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        365540593.486
Skew:                          36.486   Prob(JB):                         0.00
Kurtosis:                    1729.768   Cond. No.                     1.06e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""